- 서울시 공원

In [1]:
import numpy as np
import pandas as pd
import folium

In [2]:
df = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    

In [3]:
df = df[['공원명','면적','공원주소','X좌표(WGS84)','Y좌표(WGS84)']]
df.columns = ['공원명','면적','주소','경도','위도']
df.head(3)

,공원명,면적,주소,경도,위도
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102,127.019846,37.426449


In [4]:
df.dropna(subset=['면적','경도'], inplace=True)
df.isna().sum().sum()

0

- 면적에 따라 공원크기를 지도에 표시

In [5]:
df.면적.head(70).tail(10)

60    1349556.4㎡ 
61       1511198㎡
62       1097565㎡
63        328018㎡
64     총 1447122㎡
65       207790㎡ 
66       1018689㎡
67        954553㎡
68       1667848㎡
69         13224㎡
Name: 면적, dtype: object

In [6]:
# 총 144m 같은 글에서 '총' 지우기


In [7]:
df.dropna(subset = ['면적','경도'], inplace=True)
df.isna().sum().sum()

0

- 면적에 따라 공원크기를 지도에 표시

In [8]:
df.면적.head(10)

0         2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1                                               80683㎡
2                                            9132690m²
3                                              480994㎡
4                                             2284085㎡
5                                              8948.1㎡
6    휴양 및 편익시설 : 평의자 7개 화장실 23.18㎡ 조경시설 - 은하폭포(높이 1...
7                                                6456㎥
8    80309㎡ 배수지 시설용량 : 201.000㎥ 송수 펌프용량 : 직송급수계통 30...
9                                           1980.4㎡   
Name: 면적, dtype: object

In [9]:
df.면적[6]
df.drop([6], inplace=True)

In [10]:
# 시작할 때 있는 '총' 지우기
df.면적 =df.면적.str.replace('총 ', '')

In [11]:
# 앞에 있는 숫자만 추출하기 - 구분자 m^2, m, m^3
# 구분자가 여러개인 경우 split()
import re
re.split('[,.]', 'a,b.c')

['a', 'b', 'c']

In [12]:
df.면적 = df.면적.apply(lambda x: re.split('[m㎡㎥]', x) [0])
df.면적.head()

0    2896887
1      80683
2    9132690
3     480994
4    2284085
Name: 면적, dtype: object

In [13]:
df.면적 = df.면적.astype(float).astype(int)

In [14]:
df.head()

,공원명,면적,주소,경도,위도
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805


- 공원 크기에 따라 분류

In [15]:
# 기준: 소형 < 100000 , 중형 < 1000000 , 대형
criteria = [0, 100000, 1000000, 200000000]
labels = ['소형', '중형', '대형']
size_info = [3, 7, 15]
df['분류'] = pd.cut(df.면적, criteria, labels= labels)
df['크기'] = pd.cut(df.면적, criteria, labels= size_info)
df.head()



,공원명,면적,주소,경도,위도,분류,크기
0,남산도시자연공원,2896887,서울특별시 중구 삼일대로 231(예장동),126.990377,37.550140,대형,15
1,길동생태공원,80683,서울특별시 강동구 천호대로 1291(길동생태공원),127.154779,37.540394,소형,3
2,서울대공원,9132690,경기도 과천시 대공원광장로 102,127.019846,37.426449,대형,15
3,서울숲,480994,서울특별시 성동구 뚝섬로 273 (성수동1가),127.041798,37.543072,중형,7
4,월드컵공원,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원),126.878907,37.571805,대형,15


In [16]:
df.크기 = df.크기.astype(int)

In [17]:
df.to_csv('data/서울공원요약.csv', index=False)

In [21]:
import folium

- 공원 시각화

In [24]:
park = folium.map([37.55, 126,98], zoom_start=11)
for i in df.index:
    folium.CircleMarker(
        location=[df.위도[i], df.경도[i]],
        radius=(df.크기)[i],
        tooltip=df.공원명[i],
        popup=folium.Popup(df.주소[i], max_width=200),
        color='crimson', fill_color='crimson'
    ).add_to(park)
title_html = '<h3 align="center" style="font-size:20px">서울시 공원 현황</h3>'
park.get_root().html.add_child(folium.Element(title_html))
park


TypeError: 'module' object is not callable